In [113]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor

from catboost import CatBoostRegressor, Pool
import xgboost as xgb
import lightgbm as lgb

# Load Data

In [114]:
#Load data
df = pd.read_csv("./data/train_v9rqX0R.csv")

In [115]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# EDA and Data Cleaning

In [116]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [117]:
df.describe(include='all')

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
count,8523,7060.000000,8523,8523.000000,8523,8523.000000,8523,8523.000000,6113,8523,8523,8523.000000
unique,1559,NaN,5,NaN,16,NaN,10,NaN,3,3,4,NaN
top,FDW13,NaN,Low Fat,NaN,Fruits and Vegetables,NaN,OUT027,NaN,Medium,Tier 3,Supermarket Type1,NaN
freq,10,NaN,5089,NaN,1232,NaN,935,NaN,2793,3350,5577,NaN
mean,NaN,12.857645,NaN,0.066132,NaN,140.992782,NaN,1997.831867,NaN,NaN,NaN,2181.288914
std,NaN,4.643456,NaN,0.051598,NaN,62.275067,NaN,8.371760,NaN,NaN,NaN,1706.499616
min,NaN,4.555000,NaN,0.000000,NaN,31.290000,NaN,1985.000000,NaN,NaN,NaN,33.290000
25%,NaN,8.773750,NaN,0.026989,NaN,93.826500,NaN,1987.000000,NaN,NaN,NaN,834.247400
50%,NaN,12.600000,NaN,0.053931,NaN,143.012800,NaN,1999.000000,NaN,NaN,NaN,1794.331000
75%,NaN,16.850000,NaN,0.094585,NaN,185.643700,NaN,2004.000000,NaN,NaN,NaN,3101.296400


In [118]:
# check for missing data
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [119]:
# Observations:
# From above results, we can see that there are missing values in the dataset for Item_Weight and Outlet_Size and 0 values in Item_Visibility.

# Replace zero visibilities with NaN and mark indicator
df["Visibility_WasZero"] = (df["Item_Visibility"] == 0).astype(int)
df.loc[df["Item_Visibility"] == 0, "Item_Visibility"] = np.nan

# Impute Item_Visibility by per-item median, then global median
item_vis_median = df.groupby("Item_Identifier")["Item_Visibility"].transform("median")
df["Item_Visibility"] = df["Item_Visibility"].fillna(item_vis_median)
df["Item_Visibility"] = df["Item_Visibility"].fillna(df["Item_Visibility"].median())

# Imputation of Item_Weight by per-item median, then global median
item_weight_median = df.groupby("Item_Identifier")["Item_Weight"].transform("median")
df["Item_Weight"] = df["Item_Weight"].fillna(item_weight_median)
df["Item_Weight"] = df["Item_Weight"].fillna(df["Item_Weight"].median())

# Impute Outlet_Size by mode within (Outlet_Type, Outlet_Location_Type), then global mode
group_size_mode = df.groupby(["Outlet_Type", "Outlet_Location_Type"])["Outlet_Size"].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
df["Outlet_Size_Missing"] = df["Outlet_Size"].isna().astype(int)
df["Outlet_Size"] = df["Outlet_Size"].fillna(group_size_mode)
df["Outlet_Size"] = df["Outlet_Size"].fillna(df["Outlet_Size"].mode().iloc[0])

In [120]:
# check unique values for categorical features
for col in df.columns:
    if df[col].nunique() < 20:
        print(f"{col}: {df[col].unique()}")
        print()

Item_Fat_Content: ['Low Fat' 'Regular' 'low fat' 'LF' 'reg']

Item_Type: ['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']

Outlet_Identifier: ['OUT049' 'OUT018' 'OUT010' 'OUT013' 'OUT027' 'OUT045' 'OUT017' 'OUT046'
 'OUT035' 'OUT019']

Outlet_Establishment_Year: [1999 2009 1998 1987 1985 2002 2007 1997 2004]

Outlet_Size: ['Medium' 'Small' 'High']

Outlet_Location_Type: ['Tier 1' 'Tier 3' 'Tier 2']

Outlet_Type: ['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']

Visibility_WasZero: [0 1]

Outlet_Size_Missing: [0 1]



In [121]:
# Observations:
# Item_Fat_Content has different variations like "low fat", "lf", "regular", "reg"
# We need to standardize these values.

def clean_fat(x):
    s = str(x).strip().lower()
    if s in {"low fat", "lf"}: return "Low Fat"
    if s in {"reg", "regular"}: return "Regular"

df["Item_Fat_Content"] = df["Item_Fat_Content"].apply(clean_fat)

# Feature engineering

In [122]:
# There seems to be a relation between Item_Identifier and Item_Type, checking unique combinations of Item_Identifier and Item_Type
df[["Item_Identifier", "Item_Type"]].drop_duplicates().head(100)


,Item_Identifier,Item_Type
0,FDA15,Dairy
1,DRC01,Soft Drinks
2,FDN15,Meat
3,FDX07,Fruits and Vegetables
4,NCD19,Household
...,...,...
100,FDT28,Frozen Foods
101,FDD10,Snack Foods
102,FDW57,Snack Foods
103,DRB48,Soft Drinks


In [123]:
# Observations:
# It seems as though the 1st 2 characters of Item_Identifier might represent the Item category such as "FD" for food, "DR" for drinks and "NC" for non-consumables.

# Extract item category from ID (FD/NC/DR)
df["Item_Category"] = df["Item_Identifier"].str[:2]

In [124]:
# For non-consumables, mark fat content as Non-Consumable
df.loc[df["Item_Category"] == "NC", "Item_Fat_Content"] = "Non-Consumable"

In [125]:
# Get Outlet age from Outlet_Establishment_Year
# Taking reference year as 2013 as per the problem statement
df["Outlet_Age"] = 2013 - df["Outlet_Establishment_Year"]

In [126]:
# Get count of outlets for each item to gauge popularity of product
df["store_count"] = df.groupby("Item_Identifier")["Item_Identifier"].transform("count")

In [127]:
# Get price per gram 
df["Price_per_gram"] = df["Item_MRP"] / df["Item_Weight"]

In [128]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Visibility_WasZero,Outlet_Size_Missing,Item_Category,Outlet_Age,store_count,Price_per_gram
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,0,0,FD,14,8,26.861204
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,0,0,DR,4,6,8.153581
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,0,0,FD,14,7,8.092457
3,FDX07,19.20,Regular,0.022934,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,1,1,FD,15,6,9.484115
4,NCD19,8.93,Non-Consumable,0.013254,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,1,0,NC,26,6,6.031512


In [129]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
Visibility_WasZero             int64
Outlet_Size_Missing            int64
Item_Category                 object
Outlet_Age                     int64
store_count                    int64
Price_per_gram               float64
dtype: object

In [130]:
# Prepare features

y = df["Item_Outlet_Sales"].values
feature_cols = [
    # numeric
    "Item_Weight", "Item_Visibility", "Item_MRP", "Outlet_Age",
    "store_count",  "Visibility_WasZero", "Outlet_Size_Missing","Price_per_gram",
    # categorical
    "Item_Identifier", "Item_Fat_Content", "Item_Type", "Item_Category",
    "Outlet_Identifier", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type"
]

X = df[feature_cols].copy()

In [131]:
# Identify categorical columns and change data type to category
cat_features = X.select_dtypes(include=["object", "category"]).columns.tolist()
for c in cat_features:
    X[c] = X[c].astype("category")

In [132]:
X.dtypes

Item_Weight              float64
Item_Visibility          float64
Item_MRP                 float64
Outlet_Age                 int64
store_count                int64
Visibility_WasZero         int64
Outlet_Size_Missing        int64
Price_per_gram           float64
Item_Identifier         category
Item_Fat_Content        category
Item_Type               category
Item_Category           category
Outlet_Identifier       category
Outlet_Size             category
Outlet_Location_Type    category
Outlet_Type             category
dtype: object

In [133]:
X.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Age,store_count,Visibility_WasZero,Outlet_Size_Missing,Price_per_gram,Item_Identifier,Item_Fat_Content,Item_Type,Item_Category,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.30,0.016047,249.8092,14,8,0,0,26.861204,FDA15,Low Fat,Dairy,FD,OUT049,Medium,Tier 1,Supermarket Type1
1,5.92,0.019278,48.2692,4,6,0,0,8.153581,DRC01,Regular,Soft Drinks,DR,OUT018,Medium,Tier 3,Supermarket Type2
2,17.50,0.016760,141.6180,14,7,0,0,8.092457,FDN15,Low Fat,Meat,FD,OUT049,Medium,Tier 1,Supermarket Type1
3,19.20,0.022934,182.0950,15,6,1,1,9.484115,FDX07,Regular,Fruits and Vegetables,FD,OUT010,Small,Tier 3,Grocery Store
4,8.93,0.013254,53.8614,26,6,1,0,6.031512,NCD19,Non-Consumable,Household,NC,OUT013,High,Tier 3,Supermarket Type1


In [134]:
# Encode categorical columns for models that take only numerical input
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_enc = X.copy()

X_enc[cat_features] = enc.fit_transform(X_enc[cat_features].astype(str))
X_enc_values = X_enc.values

In [135]:
# GroupKFold CV by Outlet_Identifier to reduce leakage

groups = df["Outlet_Identifier"].astype(str)
gkf = GroupKFold(n_splits=5)

In [136]:
# Model configs
models_config = {
    "CatBoost": {
        "init": lambda: CatBoostRegressor(loss_function="RMSE",depth=4,learning_rate=0.05,l2_leaf_reg=4.0,random_seed=42, iterations=6000,eval_metric="RMSE",verbose=False,), 
        "use_cat": True
        },
    "XGBoost": {
        "init": lambda: xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05, max_depth=8, reg_lambda=4.0, random_state=42, tree_method='hist'),
        "use_cat": False
    },
    # "LightGBM": {
    # "init": lambda: lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05, max_depth=8, reg_lambda=4.0, random_state=42, n_jobs=-1), "use_cat": False
    # },
    "RandomForest": {
        "init": lambda: RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1),
        "use_cat": False
    },
}

In [137]:
# run cross-validation for each model
results = {}

for name, cfg in models_config.items():
    print(f"\nRunning CV for: {name}")
    oof = np.zeros(len(df))
    fold_rmses = []

    for fold, (trn_idx, val_idx) in enumerate(gkf.split(X, y, groups=groups), 1):
        y_trn, y_val = y[trn_idx], y[val_idx]

        if cfg['use_cat']:
            X_trn,X_val = X.iloc[trn_idx],X.iloc[val_idx]
        else:
            X_trn,X_val = X_enc_values[trn_idx],X_enc_values[val_idx]

        model = cfg['init']()

        if name == 'CatBoost':
            train_pool = Pool(X_trn, label=y_trn, cat_features=cat_features)
            valid_pool = Pool(X_val, label=y_val, cat_features=cat_features)
            model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=400, verbose=False)
            pred = model.predict(valid_pool)
        else:
            if name == 'XGBoost':
                model.fit(X_trn, y_trn, eval_set=[(X_val, y_val)], verbose=False)
            # elif name == 'LightGBM':
            #     model.fit(X_trn, y_trn, eval_set=[(X_val, y_val)])
            else:
                model.fit(X_trn, y_trn)
            pred = model.predict(X_val)

        oof[val_idx] = pred
        fold_rmse = np.sqrt(mean_squared_error(y_val, pred))
        fold_rmses.append(fold_rmse)
        print(f"  Fold {fold} RMSE: {fold_rmse:.4f}")

    cv_rmse = np.sqrt(mean_squared_error(y, oof))
    results[name] = {"oof": oof, "cv_rmse": cv_rmse, "folds": fold_rmses}
    print(f"{name} CV RMSE: {cv_rmse:.4f} | folds: {[round(x,4) for x in fold_rmses]}")

# Choose best model
best_model_name = min(results.keys(), key=lambda k: results[k]['cv_rmse'])
print(f"\nBest model by CV RMSE: {best_model_name} ({results[best_model_name]['cv_rmse']:.4f})")


Running CV for: CatBoost
  Fold 1 RMSE: 2210.9693
  Fold 2 RMSE: 1038.2119
  Fold 3 RMSE: 1136.8820
  Fold 4 RMSE: 1089.1575
  Fold 5 RMSE: 1327.8564
CatBoost CV RMSE: 1397.0126 | folds: [np.float64(2210.9693), np.float64(1038.2119), np.float64(1136.882), np.float64(1089.1575), np.float64(1327.8564)]

Running CV for: XGBoost
  Fold 1 RMSE: 2205.5582
  Fold 2 RMSE: 1149.9323
  Fold 3 RMSE: 1284.0211
  Fold 4 RMSE: 1317.7413
  Fold 5 RMSE: 1323.1619
XGBoost CV RMSE: 1478.2136 | folds: [np.float64(2205.5582), np.float64(1149.9323), np.float64(1284.0211), np.float64(1317.7413), np.float64(1323.1619)]

Running CV for: RandomForest
  Fold 1 RMSE: 1892.6306
  Fold 2 RMSE: 981.8019
  Fold 3 RMSE: 1239.3839
  Fold 4 RMSE: 1209.2471
  Fold 5 RMSE: 1235.7946
RandomForest CV RMSE: 1330.1990 | folds: [np.float64(1892.6306), np.float64(981.8019), np.float64(1239.3839), np.float64(1209.2471), np.float64(1235.7946)]

Best model by CV RMSE: RandomForest (1330.1990)


In [138]:
# since random forest was selected as the best model. In interest of compute time, we will do hyperparamter tuning on RF only
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [10,50,100,150,200],
    'max_depth': [None, 2, 5, 10],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [2,4,6,10,16],
    'bootstrap': [True, False]
}

rf = RandomForestRegressor()
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    scoring='neg_root_mean_squared_error',
    cv=GroupKFold(n_splits=5),
    verbose=0,
    random_state=42,
    return_train_score=True,
    refit=True
)

random_search.fit(X_enc_values, y, groups=groups)
print("Best parameters found:", random_search.best_params_)
print("Best CV RMSE:", -random_search.best_score_)

Best parameters found: {'n_estimators': 10, 'min_samples_split': 15, 'min_samples_leaf': 4, 'max_features': 10, 'max_depth': 5, 'bootstrap': True}
Best CV RMSE: 1232.6789856611651


In [ ]:
# Load test data
test_path = "./data/test_AbJTz2l.csv"
test_df = pd.read_csv(test_path)

# Apply same feature engineering as training data
test_df["Visibility_WasZero"] = (test_df["Item_Visibility"] == 0).astype(int)
test_df.loc[test_df["Item_Visibility"] == 0, "Item_Visibility"] = np.nan

# Impute Item_Visibility by per-item median, then global median from train
item_vis_median_test = test_df.groupby("Item_Identifier")["Item_Visibility"].transform("median")
test_df["Item_Visibility"] = test_df["Item_Visibility"].fillna(item_vis_median_test)
test_df["Item_Visibility"] = test_df["Item_Visibility"].fillna(df["Item_Visibility"].median())

# Impute Item_Weight by per-item median, then global median from train
item_weight_median_test = test_df.groupby("Item_Identifier")["Item_Weight"].transform("median")
test_df["Item_Weight"] = test_df["Item_Weight"].fillna(item_weight_median_test)
test_df["Item_Weight"] = test_df["Item_Weight"].fillna(df["Item_Weight"].median())

# Impute Outlet_Size by mode within (Outlet_Type, Outlet_Location_Type)
group_size_mode_test = test_df.groupby(["Outlet_Type", "Outlet_Location_Type"])["Outlet_Size"].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
test_df["Outlet_Size_Missing"] = test_df["Outlet_Size"].isna().astype(int)
test_df["Outlet_Size"] = test_df["Outlet_Size"].fillna(group_size_mode_test)
test_df["Outlet_Size"] = test_df["Outlet_Size"].fillna(df["Outlet_Size"].mode().iloc[0])

# Standardize Item_Fat_Content
test_df["Item_Fat_Content"] = test_df["Item_Fat_Content"].apply(clean_fat)

# Extract item category from ID (FD/NC/DR)
test_df["Item_Category"] = test_df["Item_Identifier"].str[:2]

# For non-consumables, mark fat content as Non-Consumable
test_df.loc[test_df["Item_Category"] == "NC", "Item_Fat_Content"] = "Non-Consumable"

# Get Outlet age from Outlet_Establishment_Year
test_df["Outlet_Age"] = 2013 - test_df["Outlet_Establishment_Year"]

# Get count of outlets for each item (in test set)
test_df["store_count"] = test_df.groupby("Item_Identifier")["Item_Identifier"].transform("count")

# Get price per gram
test_df["Price_per_gram"] = test_df["Item_MRP"] / test_df["Item_Weight"]

In [ ]:
# Use the best hyperparameter-tuned RandomForest model for test predictions

# Prepare test features
X_test = test_df[feature_cols].copy()
for c in cat_features:
    X_test[c] = X_test[c].astype("category")

# Encode categoricals using fitted encoder
X_test_enc = X_test.copy()
if len(cat_features) > 0:
    X_test_enc[cat_features] = enc.transform(X_test_enc[cat_features].astype(str))
X_test_vals = X_test_enc.values

# Use the best estimator from RandomizedSearchCV
best_rf = random_search.best_estimator_
test_pred = best_rf.predict(X_test_vals)

# Save predictions
submission = pd.DataFrame({
    "Item_Identifier": test_df["Item_Identifier"],
    "Outlet_Identifier": test_df["Outlet_Identifier"],
    "Item_Outlet_Sales": test_pred
})
submission.to_csv("./data/submission.csv", index=False)
print("Test predictions saved to ./data/submission.csv")

Test predictions saved to ./data/submission3.csv
